In [2]:
import pandas as pd
import glob

In [ ]:
# Get path for all tweet files to concatenate them
path = r'./elon_musk_tweets'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    # print(filename)
    frame = pd.read_csv(filename, index_col=None, encoding="utf-8")
    li.append(frame)

# print(li)
df = pd.concat(li, axis=0, ignore_index=True)
df.shape
# df[35000:35101]
# df.head(10)
# df.date

In [ ]:
df_elon = df[["date","tweet"]]
# df_elon.date[:50]
# df_elon[:10]
# df_elon.tail(3)

In [ ]:
# Saving tweets to a list
dates = df_elon["date"].to_list()
# print(dates[-1])
tweets = df_elon["tweet"].to_list()
len(tweets)

In [ ]:
import stanza
stanza.download('en') # To comment out after first usage when the model is downloaded

In [ ]:
# NER with Stanza from StanfordNLP to see it in action
# def stanza_nlp(text):
#   nlp = stanza.Pipeline(lang='en', processors='tokenize,ner')
#   doc = nlp(text)
#   print(*[f'entity: {ent.text}\ttype: {ent.type}' for sent in doc.sentences for ent in sent.ents], sep='\n')
  # print(*[f'token: {token.text}\tner: {token.ner}' for sent in doc.sentences for token in sent.tokens], sep='\n')
# stanza_nlp(tweets[100])

In [19]:
# ORG extraction with Stanza
def mentioned_orgs(tweets):
    """Function takes in a list of tweets and returns a dictionary with organisation's name as a key and a list of tweets and dates of posting as a value"""
    comps = {}
    nlp = stanza.Pipeline(lang='en', processors='tokenize,ner')
    count = 0
    for tweet in tweets:
        doc = nlp(tweet)
        for sent in doc.sentences:
            for ent in sent.ents:
                if ent.type == "ORG":
                    if ent.text not in comps:
                        comps[ent.text] = list()
                        comps[ent.text].append(list())
                        comps[ent.text][0].append(dates[count])
                        comps[ent.text][0].append(tweets[count])
                        # comps[ent.text] = 1
                    else:
                        comps[ent.text].append(list())
                        comps[ent.text][-1].append(dates[count])
                        comps[ent.text][-1].append(tweets[count])
                        # comps[ent.text] += 1
        count += 1
    
    return comps

In [ ]:
companies = mentioned_orgs(tweets)

In [ ]:
# How many times each company is mentioned
top_comps = {}
for k in companies:
    top_comps[k] = len(companies[k])
    # print(k, len(comps[k]))
    
print(top_comps)

In [ ]:
# sorting dictionary in descending order
top_comps = sorted(top_comps.items(), key=lambda x: x[1], reverse=True)
print(top_comps[:21])

In [ ]:
# comp_products = {
    # "Model S": "Tesla"
# }

comp_products = {
    "Zip2": "Zip2",
    "Falcon": "SpaceX",
    "Falcon Heavy": "SpaceX",
    "Falcon 9": "SpaceX",
    "Grasshopper": "SpaceX", 
    "Starship": "SpaceX", 
    "Merlin": "SpaceX", 
    "Kestrel": "SpaceX", 
    "Raptor": "SpaceX", 
    "Methox thruster": "SpaceX", 
    "Draco": "SpaceX", 
    "SuperDraco": "SpaceX", 
    "Dragon": "SpaceX", 
    "Cargo Dragon": "SpaceX", 
    "Starlink": "SpaceX", 
    "Hyperloop": "SpaceX", 
    "vactrain": "SpaceX",
    "Model S": "Tesla",
    "Roadster": "Tesla",
    "Model X": "Tesla", 
    "Model 3": "Tesla", 
    "Model Y": "Tesla", 
    "Semi": "Tesla", 
    "Cybertruck": "Tesla", 
    "Powerwall": "Tesla", 
    "Solar Roof": "Tesla", 
    "Solar Inverter": "Tesla", 
    "Powerpack": "Tesla", 
    "Megapack": "Tesla", 
    "Supercharger": "Tesla", 
    "Destination charger": "Tesla", 
    "Autopilot": "Tesla", 
    "Full self-driving": "Tesla", 
    "FSD": "Tesla",
    "Application-Specific Integrated Circuit": "Neuralink",
    "ASIC": "Neuralink", 
    "brain-computer interface": "Neuralink",
    "Hawthorne tunnel": "The Boring Company", 
    "Los Angeles tunnel": "The Boring Company", 
    "Loop": "The Boring Company", 
    "Loop System": "The Boring Company",
    "Gym": "OpenAI", 
    "RoboSumo": "OpenAI", 
    "Debate Game": "OpenAI", 
    "Dactyl": "OpenAI", 
    "GPT": "OpenAI", 
    "generative pre-training": "OpenAI", 
    "GPT-2": "OpenAI", 
    "GPT-3": "OpenAI", 
    "MuseNet": "OpenAI", 
    "API": "OpenAI", 
    "DALL-E": "OpenAI", 
    "CLIP": "OpenAI", 
    "Microscope": "OpenAI", 
    "Codex": "OpenAI", 
    "Five": "OpenAI", 
    "GYM Retro": "OpenAI"
}

# print(comp_products["Model S"])

In [ ]:
# Cleaning data processed with Spacy checking the entities against the names of companies' products.
new_d = {}
for k in companies:
    if k in comp_products:
        # print(k)
        # print(comp_products[k])
        # print(companies[k])
        if comp_products[k] not in new_d:
            new_d[comp_products[k]] = companies[k]
        else:
            new_d[comp_products[k]].extend(companies[k])
    else:
        if k in new_d:
            # print(k)
            new_d[k].extend(companies[k])
        else:
            # print(k)
            new_d[k] = companies[k]

# print(len(new_d["Tesla"]))

In [ ]:
# dictionary to count times each company was mentioned
new_d_count = {}
for k in new_d:
    new_d_count[k] = len(new_d[k])
    # print(k, len(elon_companies[k]))
    
# print(new_d_count)

new_d_sorted = sorted(new_d_count.items(), key=lambda x: x[1], reverse=True)
print("Companies Elon Musk most talks about:\n")
count = 1
for k,v in new_d_sorted[:31]:
    print("{}. {} ({} tweets)".format(count, k, v))
    count += 1

In [ ]:
def company_tweets_to_list(companies_dict, company_name):
    """Function takes in a dictionary with tweets about companies and a string with the name of a company under consideration"""
    comp_tweets = []

    idx = 0
    while idx in range(len(companies_dict[company_name])):
        comp_tweets.append(companies_dict[company_name][idx][1])
        idx += 1

    return comp_tweets

In [ ]:
tesla_tweets = company_tweets_to_list(new_d, company_name="Tesla")
# tesla_tweets[:2]

In [ ]:
spacex_tweets = company_tweets_to_list(new_d, company_name="SpaceX")

In [ ]:
tbc_tweets = company_tweets_to_list(new_d, company_name="The Boring Company")

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
def sentiment_analyzer(lst_texts):
    """Function takes in a list of texts and returns a dictionary with keys as numbers and text and sentiment scores with explanation for a value"""
    data = {}
    analyzer = SentimentIntensityAnalyzer()
    count = 0
    for text in lst_texts:
        vs = analyzer.polarity_scores(text)
        data[count] = list()
        data[count].extend([text, vs])
        data[count].append(data[count][1]['compound'])
        if data[count][1]['compound'] > 0:
            data[count].append("POSITIVE")
        elif data[count][1]['compound'] == 0:
            data[count].append("NEUTRAL")
        else:
            data[count].append("NEGATIVE")
        count += 1
    # print(data[3000])
    return data

In [ ]:
tesla = sentiment_analyzer(tesla_tweets)

In [ ]:
tesla_df = pd.DataFrame.from_dict(tesla, orient="index",columns=["Tweet","Scores", "Compound", "Sentiment_type"])
tesla_df

In [ ]:
tesla_df.Sentiment_type.value_counts().plot(kind='bar',title="Tesla Sentiment Analysis")

In [ ]:
spacex = sentiment_analyzer(spacex_tweets)
spacex_df = pd.DataFrame.from_dict(spacex, orient="index",columns=["Tweet","Scores", "Compound", "Sentiment_type"])
spacex_df
spacex_df.Sentiment_type.value_counts().plot(kind='bar',title="SpaceX Sentiment Analysis", color="#334CFF")

In [ ]:
tbc = sentiment_analyzer(tbc_tweets)
tbc_df = pd.DataFrame.from_dict(tbc, orient="index",columns=["Tweet","Scores", "Compound", "Sentiment_type"])
tbc_df
tbc_df.Sentiment_type.value_counts().plot(kind='bar',title="Sentiment Analysis for The Boring Company",color="#FF338A")

In [ ]:
apple_tweets = company_tweets_to_list(new_d, company_name="Apple")
# apple_tweets[:10]

In [ ]:
apple = sentiment_analyzer(apple_tweets)
apple_df = pd.DataFrame.from_dict(apple, orient="index",columns=["Tweet","Scores", "Compound", "Sentiment_type"])
apple_df
apple_df.Sentiment_type.value_counts().plot(kind='bar',title="Apple Sentiment Analysis",color="#35950E")

In [ ]:
google_tweets = company_tweets_to_list(new_d, company_name="Google")
google = sentiment_analyzer(google_tweets)
google_df = pd.DataFrame.from_dict(google, orient="index",columns=["Tweet","Scores", "Compound", "Sentiment_type"])
google_df
google_df.Sentiment_type.value_counts().plot(kind='bar',title="Google Sentiment Analysis",color="#DE3A20")

In [ ]:
twitter_tweets = company_tweets_to_list(new_d, company_name="Twitter")
twitter = sentiment_analyzer(twitter_tweets)
twitter_df = pd.DataFrame.from_dict(twitter, orient="index",columns=["Tweet","Scores", "Compound", "Sentiment_type"])
twitter_df
twitter_df.Sentiment_type.value_counts().plot(kind='bar',title="Twitter Sentiment Analysis",color="#51CEE5")